# Final Project: Regression Analysis (Medical Cost Dataset)

### **Author:** Deb St. Cyr
### **Date:** November 2025


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set(style="whitegrid")

# Load your uploaded dataset
df = pd.read_csv("data/insurance.csv")

# Display first 10 rows
df.head(10)

In [ ]:
import os
os.getcwd()
